<a href="https://colab.research.google.com/github/nembangallen/Applied-Data-Science-With-Python/blob/master/Stock_market_predicctionipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Market Prediction And Forecasting Using Stacked LSTM

In [ ]:
### Keras and Tensorflow >2.0

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
def pre_processing(df1):
  '''Data Processing modeule
  1. take only closing value from dataframe
  2. scale value from 0 to 1 taking minimum and maxixum value from closing value
  3. split to train and test 70-30 %
  4. return train and test data
  '''
  df1=df.reset_index()['c']   #read only closing value
  ### LSTM are sensitive to the scale of the data. so we apply MinMax scaler 
  scaler=MinMaxScaler(feature_range=(0,1))
  df1=scaler.fit_transform(np.array(df1).reshape(-1,1))
  ##splitting dataset into train and test split
  training_size=int(len(df1)*0.65)
  test_size=len(df1)-training_size
  train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]
  # reshape into X=t,t+1,t+2,t+3 and Y=t+4
  time_step = 100
  X_train, y_train = create_dataset(train_data, time_step)
  X_test, ytest = create_dataset(test_data, time_step)
  # reshape input to be [samples, time steps, features] which is required for LSTM
  X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
  return X_train,y_train, X_test,ytest,test_data

In [ ]:
# Build LSTM
def Build_model():
  ''' LSTM Module
      1. create sequential model
      2. add 3 lstm layer with 50 lstm node
      3. add dense layer with 1 node as output
      4. return model
  '''
  model=Sequential()
  model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
  model.add(LSTM(50,return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',optimizer='adam')
  return model


In [ ]:
# Train model
def train_model(model,X_train,y_train,X_test,ytest,epochs):
  '''This module is for training the model'''
  model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=epochs,batch_size=64,verbose=1)
  return model

In [ ]:
### Lets Do the prediction and check performance metrics
def test_model(model,X_train,X_test):
  train_predict=model.predict(X_train)
  test_predict=model.predict(X_test)
  ##Transformback to original form
  train_predict=scaler.inverse_transform(train_predict)
  test_predict=scaler.inverse_transform(test_predict)
  return train_predict, test_predict

In [ ]:
#forcasting
def forcase_data(model,test_data,n_steps=100):
  x_input=test_data[len(test_data)-100:].reshape(1,-1)
  temp_input=list(x_input)
  temp_input=temp_input[0].tolist()
  # demonstrate prediction for next 10 days
  lst_output=[]
  i=0
  while(i<7):
      if(len(temp_input)>100):
          #print(temp_input)
          x_input=np.array(temp_input[1:])
          print("{} day input {}".format(i,x_input))
          x_input=x_input.reshape(1,-1)
          x_input = x_input.reshape((1, n_steps, 1))
          #print(x_input)
          yhat = model.predict(x_input, verbose=0)
          print("{} day output {}".format(i,yhat))
          temp_input.extend(yhat[0].tolist())
          temp_input=temp_input[1:]
          #print(temp_input)
          lst_output.extend(yhat.tolist())
          i=i+1
      else:
          x_input = x_input.reshape((1, n_steps,1))
          yhat = model.predict(x_input, verbose=0)
          print(yhat[0])
          temp_input.extend(yhat[0].tolist())
          print(len(temp_input))
          lst_output.extend(yhat.tolist())
          i=i+1
  return lst_output
      


In [ ]:
def Save_Model(model,path,model_name):
  # serialize model to JSON
  save_to_json = model_name+'.json'
  save_to_h5 = model_name+'.h5'
  model_json = model.to_json()
  with open(save_to_json, "w") as json_file:
      json_file.write(model_json)
  # serialize weights to HDF5
  model.save_weights(save_to_h5)
  print("Saved model {} to disk".format(model_name))

In [ ]:
PATH = os.getcwd()
nepsec_data_path = PATH + '/nepsedata' #give SOURCE  data  PATH
nepsec_trainned_path = PATH + '/nepsecModels' 
dataname_list = []
data_list = []
for x in os.listdir(nepsec_data_path):
    if x.endswith(".csv"):
      dataname_list.append(x)
      df=pd.read_csv(nepsec_data_path+'/'+x)
      print("reading data {} complete".format(x))
      data_list.append(df)
      X_train,y_train, X_test,ytest,test_data = pre_processing(df1=df)
      model = Build_model()
      model = train_model(model=model,X_train=X_train,y_train=y_train,X_test=X_test,ytest=ytest,epochs=1)
      model_name = x.split(".")
      Save_Model(model=model,path=nepsec_trainned_path,model_name=model_name[0])
      # train_predict, test_predict = test_model(model=model,X_train=X_train,X_test=X_test)
      # lst_output = forcase_data(model=model,test_data=test_data,n_steps=100)
      # print(lst_output)
    else:
      pass

